In [ ]:
import re
import sys
import git
import uuid
import jellyfish
import pandas as pd
import numpy as np
from pathlib import Path 
from scipy.special import softmax
from tqdm.notebook import trange

In [ ]:
# git for functions loading and work path finding
import git
repo = git.Repo('.', search_parent_directories=True)
work_path = Path(repo.working_tree_dir)
if str(work_path) not in sys.path:
    sys.path.append(str(work_path))

In [ ]:
from function.seqfilter import SeqFilter
from function.utilities import fasta_to_seqlist
from function.utilities import get_fasta_filename

In [ ]:
def get_homology_frag(fasta_path):
    '''
    TODO: 跟老師的part依樣part就好
    '''
    
    # get oma_group_id and all homologs 
    fasta_path = Path(fasta_path)
    oma_group_id = get_fasta_filename(fasta_path)
    seq_list = fasta_to_seqlist(fasta_path)

    # filter sequence length and add the gap to od_ident
    od_ident = pondr_df[pondr_df['oma_group_id'] == oma_group_id]['od_ident'].tolist()[0]
    od_ident = seqfilter.length_filter_by_od_ident(od_ident,disorder_filter_length,order_filter_length)
    od_ident = seqfilter.od_add_alignment(fasta_path,od_ident)   
    od_index = seqfilter.get_od_index(od_ident)['disorder_region']
    
    # create id for sub-homologs
    homo_frag_seq_id = []
    for i in od_index:
        homo_frag_seq_id.append(uuid.uuid4().hex)

    # process for getting 
    seq_frag_list = []
    
    # loop for a homolog containig many species sequences from a fasta file
    for seq in seq_list: 
        
        # loop for sub-homologs referenced by the gap added od_ident
        for index,element in enumerate(od_index):
            start = element['start']
            end = element['end']
        
            # get some infos
            oma_protein_id = seq.id
            homology_id = homo_frag_seq_id[index]
            frag_seq_id = uuid.uuid4().hex 

            # get frag sequence by index from od_ident
            frag_seq = str(seq.seq)[start:end]
            
            # remove seqeunce's gap, sequence length is shorter than disorder_filter_length will not be added to our dataset
            frag_seq = frag_seq.replace("-","") 
            if len(frag_seq) < disorder_filter_length:
                continue
             
            # remove amino acid "M" if at the start of the sequence
            if frag_seq[0] == "M" and start == 0: 
                frag_seq = frag_seq[1:]
                  
            # mark human sequence for calculating sample probability later
            human_seq_match = re.match('^HUMAN*', oma_protein_id)
            if human_seq_match:
                is_human_seq = 1
            else:
                is_human_seq = 0
                
            # misc
            seq_frag_list.append({"oma_group_id":oma_group_id,
                                  "oma_protein_id":oma_protein_id,
                                  "homology_id":homology_id,
                                  "frag_seq_id":frag_seq_id,
                                  "frag_seq_len":len(frag_seq),
                                  "frag_seq":frag_seq,
                                  "is_human_seq":is_human_seq,
                                 })
            
    return pd.DataFrame(seq_frag_list)

In [ ]:
seqfilter = SeqFilter()
disorder_filter_length = 40 
order_filter_length = 10

# read sequence alignment fasta path (d_alignment)
fasta_path = work_path / "1_prepare_training_data" / "oma_all" / "d_alignment" / "7711_alied"
fasta_path_list = list(fasta_path.rglob("*.fasta"))

# read pondr identified order/disorder info
pondr_df = pd.read_pickle(work_path / "1_prepare_training_data" / "1-2_human_pondr_vsl2.pkl") 

In [ ]:
# make whole dataset by looping alignment fasta files
df = pd.DataFrame()
t = trange(len(fasta_path_list), desc='', leave=True, position=0)
failed_list = []
for i in t:
    oma_id = get_fasta_filename(fasta_path_list[i])
        
    df = pd.concat([df,
                    get_homology_frag(fasta_path_list[i])],ignore_index=True)

In [ ]:
# get q_prob for sampling probaility during training
human_df = df[df["oma_protein_id"].str.match(r'^HUMAN*')==True].reset_index(drop=True) 
def get_sim_to_human_seq(x):
    human_seq = human_df[human_df['homology_id'] == x['homology_id']]['frag_seq'].tolist()[0]
    sim_score = jellyfish.levenshtein_distance(human_seq, x['frag_seq'])
    return sim_score
df['q_prob_score'] = df.apply(get_sim_to_human_seq, axis=1)
df['q_prob_pure'] = df.groupby('homology_id')["q_prob_score"].transform(lambda x : x/sum(x))

In [ ]:
# deprecated
# df["q_prob_softmax"] = df.groupby('homology_id')["q_prob_score"].transform(softmax) 
# deprecated, get k_prob, 
# homologs_group_counts =  df.groupby('homology_id')['homology_id'].transform('count').values
# homologs_group_nums = df['homology_id'].unique().shape[0] 
# df['k_prob'] = (1/homologs_group_nums)/homologs_group_counts 

In [ ]:
# save to pickle
# df.to_pickle(work_path / "1_prepare_training_data" / "1-3_vsl2_omaseq_with_prob.pkl")